In [2]:
import os
import sys

from bids import BIDSLayout
import numpy as np
import pandas as pd
import pickle

from deepmreye import architecture, train, analyse, preprocess
from deepmreye.util import util, data_generator, model_opts

from plotly.offline import init_notebook_mode
from IPython.core.display import display, HTML

# Initialize plotly figures
init_notebook_mode(connected=True)

# Make sure the output width is adjusted for better export as HTML
display(HTML("<style>.container { width:70% !important; }</style>"))
display(HTML("<style>.output_result { max-width:70% !important; }</style>"))

# Change to os.environ["CUDA_VISIBLE_DEVICES"] = "" if you dont have access to a GP
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Autoreload modules
%load_ext autoreload
%autoreload 2

# lint with black
%load_ext lab_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [4]:
model_weights = "../inputs/models/"
experiment_folder = "../inputs/rest_blnd_can_fmriprep/"

task = "rest"
suffix = "bold"
extension = "nii.gz"
space = "MNI152NLin2009cAsym"

debug = True

In [5]:
layout = BIDSLayout(experiment_folder, validate=False, derivatives=False)

In [7]:
# Preload masks to save time within participant loop
(eyemask_small, eyemask_big, dme_template, mask, x_edges, y_edges, z_edges) = preprocess.get_masks()

print(eyemask_small)

subjects = layout.get(return_type='id', target='subject')

if debug:
    subjects = [subjects[0]]
    
for sub in subjects:
    print(f"Running participant: {sub}")
    bf = layout.get(return_type='filename', subject=sub, suffix=suffix, task=task, extension=extension, space=space)
    print(f"Input file: {bf}")
    preprocess.run_participant(fp_func, dme_template, eyemask_big, eyemask_small, x_edges, y_edges, z_edges)

ValueError: File ../deepmreye/masks/eyemask_small.nii does not exist!

In [ ]:
# Preload masks to save time within participant loop
(eyemask_small, eyemask_big, dme_template, mask, x_edges, y_edges, z_edges) = preprocess.get_masks()

# Loop across participants and extract eye mask
for participant in participants:
    if participant.startswith('s'):
        print('Running participant {}'.format(participant))
        participant_folder = functional_data + participant
        for run in os.listdir(participant_folder):
            if run.startswith('run'):
                fp_func = participant_folder + os.path.sep + run # Filepath to functional
                preprocess.run_participant(fp_func, dme_template, eyemask_big, eyemask_small, x_edges, y_edges, z_edges)

Note that the pipeline produces diagnostic figures that can and should be carefully examined for each individual scanning run. These figures are saved as interactive html file together with the rest of the model output. Here is one such [html file](https://github.com/DeepMReye/DeepMReye/blob/main/tests/data/report_test_participant.html) as an example. If the alignment is off, try running above loop with:
```python
preprocess.run_participant(fp_func, dme_template, eyemask_big, eyemask_small, x_edges, y_edges, z_edges, transforms=['Affine', 'Affine', 'SyNAggro'])
```
which uses a more aggressive (and non-linear) alignment procedure. 

In [ ]:
# Combine processed masks with labels
for participant in participants:
    if participant.startswith('s'):
        print('Running participant {}'.format(participant))
        participant_folder = functional_data + participant
        participant_data, participant_labels, participant_ids = [], [], []
        for run_idx, run in enumerate(os.listdir(participant_folder)):
            if not run.endswith(".p"):
                continue
            # Load mask and normalize it
            this_mask = participant_folder + os.path.sep + run
            this_mask = pickle.load(open(this_mask, 'rb'))
            this_mask = preprocess.normalize_img(this_mask)
        
            # If experiment has no labels use dummy labels
            this_label = np.zeros((this_mask.shape[3], 10, 2)) # 10 is the number of subTRs used in the pretrained weights, 2 is XY
        
            # Check if each functional image has a corresponding label. Note that mask has time as third dimension
            if not this_mask.shape[3] == this_label.shape[0]:
                print('WARNING --- Skipping Subject {} Run {} --- Wrong alignment (Mask {} - Label {}).'.format(subj, run_number, this_mask.shape, this_label.shape))
                continue
        
            # Store across runs
            participant_data.append(this_mask)
            participant_labels.append(this_label)
            participant_ids.append(([participant]*this_label.shape[0], [run_idx]*this_label.shape[0]))
    
        # Save participant file
        preprocess.save_data(participant + 'no_label', participant_data, participant_labels, participant_ids, processed_data, center_labels=False)

---
<a name="load"></a>
## Load & visualize input

Here, we showcase the input / output pairs that DeepMReye uses to decode position from the eye balls of the participant by visualizing the input of the model in the left panel ('Normalized MR-Signal') and the training labels in the right panel ('Gaze position').

In [ ]:
# Define paths to example dataset
datasets = [processed_data + p for p in os.listdir(processed_data) if 'no_label' in p]

# Load data from one participant to showcase input/output
X, y = data_generator.get_all_subject_data(datasets[0])
print('Input: {}, Output: {}'.format(X.shape, y.shape))

In [ ]:
fig = analyse.visualise_input_data(X, y, bg_color="rgb(255,255,255)", ylim=[-11, 11])
fig.show()

The figure above shows a slice through the coregistered eyeballs of an exemplary participant (left panel) as well as the corresponding horizontal (X) and vertical (Y) gaze position over time. The normalized MRI signal was color coded (red = strong, black = weak MRI signal relative to baseline). The slider at the bottom allows exploring the eyeball-MRI signal and its relationship to gaze position over time (i.e. functional images).

---
<a name="train"></a>
## Load model weights
Here we use a pretrained model to decode gaze position from the example dataset. Model weights for all datasets and combined model runs can be downloaded under model_weights here: [https://osf.io/mrhk9/](https://osf.io/mrhk9/)

In [ ]:
opts = model_opts.get_opts()
test_participants = [processed_data + p for p in os.listdir(processed_data) if 'no_label' in p]
generators = data_generator.create_generators(test_participants, test_participants)
generators = (*generators, test_participants, test_participants) # Add participant list

In [ ]:
model_weights = model_weights + 'datasets_1to5.h5'

# Get untrained model and load with trained weights
(model, model_inference) = train.train_model(dataset='example_data', generators=generators, opts=opts, return_untrained=True)
model_inference.load_weights(model_weights)

---
<a name="evaluate"></a>
## Model evaluation

We can now evaluate all participants on the trained weights. 
Verbosity can be set here to get a more fine-grained detail of the model performance over different evaluation metrics (Pearson, [R^2-Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html), Euclidean Error). We then plot the group results for different metrics as a boxplot (left, hovering across single dots indicates participant identifier) and show the real and predicted gaze position as a line plot to the right split into X and Y component. 

In [ ]:
(evaluation, scores) = train.evaluate_model(dataset='example_data', model=model_inference, generators=generators,
                                            save=False, model_path=experiment_folder, model_description='', verbose=2, percentile_cut=80)

In [ ]:
fig = analyse.visualise_predictions_slider(evaluation, scores, color="rgb(0, 150, 175)", bg_color="rgb(255,255,255)", ylim=[-11, 11])
fig.show()